In [1]:
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import gzip
import torch

sys.path.append("../src/")
from loss_function import cs
from meter import AUCMeter
import data_reader as dr

In [2]:
corpus_path = '../data_Android/corpus.tsv/corpus.txt'
features = dr.build_tfidf_features(corpus_path)


In [3]:
path_pos = '../data_Android/dev.pos.txt'
path_neg = '../data_Android/dev.neg.txt'
qIDsDev, qCandidatesDev, labelsDev = dr.build_android_qsets(path_pos, path_neg)

path_pos = '../data_Android/test.pos.txt'
path_neg = '../data_Android/test.neg.txt'
qIDsTest, qCandidatesTest, labelsTest = dr.build_android_qsets(path_pos, path_neg)


In [4]:
def computeAUC(qIDs, qCandidates, labels):
    scores = []
    targets = []
    
    for q in qIDs:
        qTensor = features[q]
        for p in qCandidates[q]:
            pTensor = features[p]
            score = cs(qTensor, pTensor, 0)
            scores.append(score)
        targets.extend(labels[q])
    
    aucmeter = AUCMeter()
    aucmeter.reset()
    output = torch.cat(scores) 
    expect = torch.LongTensor(targets)
    aucmeter.add(output, expect)
    
    return aucmeter.value(max_fpr=0.05)

In [5]:
AUC_dev = computeAUC(qIDsDev, qCandidatesDev, labelsDev)
AUC_test = computeAUC(qIDsTest, qCandidatesTest, labelsTest)

In [6]:
print ('AUC(0.05) of the dev  set: ' + str(AUC_dev))
print ('AUC(0.05) of the test set: ' + str(AUC_test))


AUC(0.05) of the dev  set: 0.70691133468
AUC(0.05) of the test set: 0.73925406644
